In [1]:
import cv2
from setup import out_dir, data_dir, image_dir, model_dir
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import util_image

%matplotlib inline

In [19]:
image_list = glob.glob(image_dir+"zoom15/*.png")
# image_list = glob.glob('/dreambig/qingyi/image_chicago/data/images/streetview/*.jpg')

In [3]:
print(len(image_list))

31760


# Re-package images for faster I/O

In [22]:
image_df = [i.split('/')[-1][:-4].split('_')+[i] for i in image_list]
image_df = pd.DataFrame(image_df, columns = ['state','county','tract','idx','dir'])
for i in ['state','county','tract','idx']:
    image_df[i] = image_df[i].astype(int)
image_df = image_df.sort_values(by=['state','county','tract','idx'])

In [24]:
image_count = image_df.groupby(['state','county','tract'], as_index=False).count()
image_count[image_count['idx']<20]

,state,county,tract,idx,dir


In [25]:
# package into 1 file
image_index = np.arange(len(image_df))
image_group = image_index // 2000
image_df.loc[:,'group'] = image_group
image_df.to_csv(data_dir+"data_info.csv", index=False)


In [26]:
# package into train/test (stratified)
train_image_df = image_df[image_df['idx'].astype(int) < 18]
test_image_df = image_df[image_df['idx'].astype(int) >= 18]

image_index = np.arange(len(train_image_df))
image_group = image_index // 2000
train_image_df.loc[:,'group'] = image_group

image_index = np.arange(len(test_image_df))
image_group = image_index // 2000
test_image_df.loc[:,'group'] = image_group

train_image_df.to_csv(data_dir+"train_data_info.csv", index=False)
test_image_df.to_csv(data_dir+"test_data_info.csv", index=False)

<ipython-input-26-89d7eb36d078>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_image_df.loc[:,'group'] = image_group
<ipython-input-26-89d7eb36d078>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_image_df.loc[:,'group'] = image_group


In [27]:
from PIL import Image
import h5py
import torchvision

In [28]:
transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.CenterCrop(256),
            torchvision.transforms.Resize(64)])

In [32]:
for (s,g),df in image_df.groupby(['state', 'group']):
    save_hdf = h5py.File(image_dir+'zoom15/hdf5/'+str(s)+"_"+str(g)+".hdf5", 'w')
    imgs = []
    for i in df['dir']:
        imgs.append(transform(util_image.center_crop(cv2.imread(i), (256,256))).numpy())
    imgs = np.array(imgs)
    dset = save_hdf.create_dataset('default', data=imgs)
    save_hdf.close()

In [33]:
for (s,g),df in test_image_df.groupby(['state', 'group']):
    save_hdf = h5py.File(image_dir+'zoom15/hdf5/test_'+str(s)+"_"+str(g)+".hdf5", 'w')
    imgs = []
    for i in df['dir']:
        imgs.append(transform(util_image.center_crop(cv2.imread(i), (256,256))).numpy())
    imgs = np.array(imgs)
    dset = save_hdf.create_dataset('default', data=imgs)
    save_hdf.close()

In [34]:
for (s,g),df in train_image_df.groupby(['state', 'group']):
    save_hdf = h5py.File(image_dir+'zoom15/hdf5/train_'+str(s)+"_"+str(g)+".hdf5", 'w')
    imgs = []
    for i in df['dir']:
        imgs.append(transform(util_image.center_crop(cv2.imread(i), (256,256))).numpy())
    imgs = np.array(imgs)
    dset = save_hdf.create_dataset('default', data=imgs)
    save_hdf.close()


# Crop

In [28]:
for i in image_list:
    im = cv2.imread(i)
    im = util_image.center_crop(im, (224,224))
    im = cv2.resize(im, (64,64))
#     plt.figure(figsize=(2,2))
#     plt.imshow(im)
#     break
    new_name = i.replace('streetview', 'streetview_64')
    cv2.imwrite(new_name, im)


# Blurring

In [7]:
for i in image_list:
    im = cv2.imread(i)
#     im_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

#     im_gaussian = cv2.GaussianBlur(im, (5,5), 5)
#     im_avg = cv2.blur(im, (5,5))

    
#     sharpen_kernel = np.array([[0, -1, 0],
#                        [-1, 5,-1],
#                        [0, -1, 0]])
#     edge_kernel = np.array([[-1, -1, -1],
#                        [-1, 8,-1],
#                        [-1, -1, -1]])    
    
#     im_sharp = cv2.filter2D(src=im, ddepth=-1, kernel=sharpen_kernel)
    
#     im_edge = cv2.GaussianBlur(im_gray, (5,5), 5)
#     im_edge = cv2.GaussianBlur(im_edge, (5,5), 5)
#     im_edge = cv2.filter2D(src=im_edge, ddepth=-1, kernel=edge_kernel)
    
    im_bilateral = cv2.bilateralFilter(src=im, d=7, sigmaColor=20, sigmaSpace=200)
    for j in range(5):
        im_bilateral = cv2.bilateralFilter(src=im_bilateral, d=7, sigmaColor=20, sigmaSpace=200)

#     plt.imshow(im_bilateral)
#     plt.imshow(im_avg)
#     plt.imshow(im_gaussian)
#     plt.imshow(im_sharp)
#     plt.imshow(im_edge, cmap='gray', vmin=0, vmax=255)
#     plt.show()
    
    new_name = i.replace('zoom15', 'zoom15_bilateral')
    cv2.imwrite(new_name, im_bilateral)
    


In [ ]:
import cv2

geoid = '17_43_846523'
images_list = [cv2.imread(image_dir+zoomlevel+"/"+geoid+"_"+str(i)+".png") for i in range(10)]
util_plot.plot_images(1, 10, images_list, mag=3);